In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json

In [3]:
url="https://www.usnews.com/education/best-global-universities/search?format=json&page=5"
my_headers = {'user-agent': 'Mozilla/5.0'}
jsonData=requests.get(url,headers=my_headers)
print(jsonData.status_code)
data=json.loads(jsonData.text)
    #print(data)
print(json.dumps(data,indent=2,ensure_ascii=False))

200
{
  "total_count": 2005,
  "total_pages": 201,
  "current_page": 5,
  "header": {
    "search": {
      "search_engine_id": "009692862532464842631:7x4cnqttvvy",
      "search_box_placeholder_text": "Search U.S. News Education",
      "id": "0000016e-4c4c-df8c-a9ee-de7ef6e50000",
      "permalink": "https://www.usnews.com/search/education"
    },
    "mega_menu": {
      "search_template_path": "templates/pages/education/higher-education/nav-search-switcher.nunj",
      "permalink": "",
      "menu": {
        "children": [
          {
            "link": {
              "pull_short_headline": false,
              "new_window": false,
              "content_reference_id": "00000142-928b-d33c-abc6-ff9fe7600000",
              "text": "Home",
              "permalink": "",
              "url": "https://www.usnews.com/education"
            },
            "permalink": "",
            "url_patterns": []
          },
          {
            "link": {
              "pull_short_headline": 

In [4]:
#直接從QS各大學網站抓取大學基本資料
def get_BaseInfo(school):
    r = requests.get("https://www.topuniversities.com/universities/"+school)
    soup = BeautifulSoup(r.text,"html.parser") #將網頁資料以html.parser

    name = soup.select("div.programeTitle h1") #取HTML標中的 <div class="title"></div> 中的<a>標籤存入sel
    loc = soup.select("div.uni-det")
    qs_rank = soup.select("div.cont")
    arr=list([name[0].text,qs_rank[0].text])
    
    baseinfo = soup.select("span.info-setails") #取HTML標中的 <div class="title"></div> 中的<a>標籤存入sel
    for s in baseinfo:
        arr.append(s.text)
    if ('Yes' not in arr) & ('No' not in arr):
        arr.insert(5,"")
    #print(arr)
    data=pd.DataFrame([arr],columns=['University','QS World University Rankings','Status', 'Research Output', 'Student/Faculty Ratio','Scholarships','International Students','Size','Total Faculty'])
    return data

In [5]:
#直接從QS各大學網站抓取大學師生資料
def get_Stundent_Staff_Info2(school):
    r = requests.get("https://www.topuniversities.com/universities/"+school)
    soup = BeautifulSoup(r.text,"html.parser") #將網頁資料以html.parse
    students_staff = soup.select("div.set div") #取HTML標中的 <div class="title"></div> 中的<a>標籤存入sel
    students_staff=students_staff[36:]
    
    name = soup.select("div.programeTitle h1")
    print(name[0].text)
    pre5=""
    pre4=""
    pre3=""
    pre2=""
    pre=""
    
    Total_students=""
    PG_students_total=""
    UG_students_total=""
    International_students=""
    PG_students_intl=""
    UG_students_intl=""
    Total_faculty_staff=""
    Domestic_staff=""
    International_staff=""
    for term in students_staff:
        s=str.strip(term.text)
        if pre == "Total students":
            Total_students=s
        elif pre == "International students":
            International_students=s
        elif pre == "Total faculty staff":
            Total_faculty_staff=s
        elif pre == "Domestic staff":
            Domestic_staff=s
        elif pre == "Int'l staff":
            International_staff=s
        elif pre[:11] == "PG students":
            if pre3=="Total students":
                PG_students_total=s
            else:
                PG_students_intl=s
        elif pre[:11] == "UG students":
            if pre3=="Total students":
                UG_students_total=s
            else:
                UG_students_intl=s
        
        #保留資訊
        pre5=pre4
        pre4=pre3
        pre3=pre2
        pre2=pre
        pre=s
    temp=pd.DataFrame([[name[0].text,Total_students,PG_students_total,UG_students_total,International_students,PG_students_intl,UG_students_intl,Total_faculty_staff,Domestic_staff,International_staff]],columns=['University','Total students','rate of PG students(Total)','rate of UG students(Total)','International students','rate of PG students(International)','rate of UG students(International)','Total faculty staff','Domestic staff','International staff'])
    print(temp)
    return temp

In [6]:
#取得網站資料(script中內容json)來源位置
def get_json_web(web):
    #index:21(subject)、20(Graduate Employability)
    r = requests.get(web)
    soup = BeautifulSoup(r.text,"html.parser") #將網頁資料以html.parse
    name = soup.select("script")
    for term in name:
        if term.text.find('"rank_indicators_url":"https:\/\/www.topuniversities.com\/sites\/default\/files\/qs-rankings-data\/en\/')>-1:
            s=term
            break
    json_web='https://www.topuniversities.com/sites/default/files/qs-rankings-data/en'+s.text[s.text.find('"rank_indicators_url":"https:\/\/www.topuniversities.com\/sites\/default\/files\/qs-rankings-data\/en\/')+102:s.text.find(',"compare_url"')-1]
    return json_web

In [7]:
#從QS大學排名網站取得資料來源(get_json_web())後，抓取原始json檔，搜索資料____大學總排名
def get_rank(year):
#Rank
    url=get_json_web('https://www.topuniversities.com/university-rankings/world-university-rankings/'+str(year))
    
    jsonData=requests.get(url)
    data=json.loads(jsonData.text)
    data=data['data']
    #print(data)
    #print(json.dumps(data,indent=2,ensure_ascii=False))

    all_school=pd.DataFrame()
    columns_name=["Overall Rank","OverallRank_Dis","University","Region","Location","City","Overall Score","International Students Ratio Score","International Faculty Ratio Score","Faculty Student Ratio Score","Citations per Faculty Score","Academic Reputation Score","Employer Reputa Score","Uni_Herf"]
    columns_name2019=["Overall Rank","OverallRank_Dis","University","Region","Location","City","Overall Score","Academic Reputation Score","Employer Reputation Score","Faculty Student Score","International Faculty Score","International Students Score","Citations per Faculty Score","Uni_Herf"]
    if year==2019:
        columns_name=columns_name2019
    for i in range(0,len(data)):
        #print(i)
        Region=data[i]["region"]
        Location=data[i]["location"]
        City=data[i]["city"]
        Overall_Rank=data[i]["overall_rank"]
        University=data[i]["uni"][data[i]["uni"].find('" class="uni-link">')+19:data[i]["uni"].find('</a>')]
        Uni_Herf=data[i]["uni"][68:data[i]["uni"].find('" class="uni-link">')]
        Overall_Score=data[i]["overall"][45:data[i]["overall"].find('</div></div>')]
        if year==2019:
            Overall_Rank_Dis=data[i]["overall_rank_dis"][45:data[i]["overall_rank_dis"].find('</div></div>')]
            Academic_Reputation=data[i]["ind_76"][45:data[i]["ind_76"].find('</div></div>')]
            Employer_Reputation=data[i]["ind_77"][45:data[i]["ind_77"].find('</div></div>')]
            Faculty_Student=data[i]["ind_74"][45:data[i]["ind_74"].find('</div></div>')]
            International_Students=data[i]["ind_73"][45:data[i]["ind_73"].find('</div></div>')]
            Citations_per_Faculty=data[i]["ind_75"][45:data[i]["ind_75"].find('</div></div>')]
            International_Faculty=data[i]["ind_37"][45:data[i]["ind_37"].find('</div></div>')]
            temp=pd.DataFrame([[Overall_Rank,Overall_Rank_Dis,University,Region,Location,City,Overall_Score,Academic_Reputation,Employer_Reputation,Faculty_Student,International_Faculty,International_Students,Citations_per_Faculty,Uni_Herf]],columns=columns_name)
        else:
            Overall_Rank_Dis=data[i]["overall_rank_dis"][45:data[i]["overall_rank_dis"].find('</div></div>')]
            Academic_Reputation_Score=data[i]["ind_76"][45:data[i]["ind_76"].find('</div></div>')]
            Employer_Reputa_Score=data[i]["ind_77"][45:data[i]["ind_77"].find('</div></div>')]
            Faculty_Student_Ratio_Score=data[i]["ind_36"][45:data[i]["ind_36"].find('</div></div>')]
            Citations_per_Faculty_Score=data[i]["ind_73"][45:data[i]["ind_73"].find('</div></div>')]
            International_Faculty_Ratio_Score=data[i]["ind_18"][45:data[i]["ind_18"].find('</div></div>')]
            International_Students_Ratio_Score=data[i]["ind_14"][45:data[i]["ind_14"].find('</div></div>')]
            temp=pd.DataFrame([[Overall_Rank,Overall_Rank_Dis,University,Region,Location,City,Overall_Score,International_Students_Ratio_Score,International_Faculty_Ratio_Score,Faculty_Student_Ratio_Score,Citations_per_Faculty_Score,Academic_Reputation_Score,Employer_Reputa_Score,Uni_Herf]],columns=columns_name)
        
        all_school=all_school.append(temp)

    #print(all_school)
    return all_school

In [8]:
#從QS大學排名網站取得資料來源(get_json_web())後，抓取原始json檔，搜索資料____學科排名
def get_sub_rank(web):
    json_web=get_json_web(web)
    
    jsonData=requests.get(json_web)
    data=json.loads(jsonData.text)
    data=data['data']
    all_school=pd.DataFrame()
    columns_name=["OverallRank","OverallRank_Dis","University","Region","Location","City","Overall Score","H-index Citations","Citations per Paper","Academic Reputation","Employer Reputation"]
    
    #print(data)
    for i in range(0,len(data)):
        Region=data[i]["region"]
        Location=data[i]["location"]
        City=data[i]["city"]
        Overall_Rank=data[i]["overall_rank"]
        Overall_Rank_Dis=data[i]["overall_rank_dis"][45:data[i]["overall_rank_dis"].find('</div></div>')]
        University=data[i]["uni"][data[i]["uni"].find('" class="uni-link">')+19:data[i]["uni"].find('</a>')]
        Overall_Score=data[i]["overall"][45:data[i]["overall"].find('</div></div>')]
        
        H_index_Citations=data[i]["ind_69"][45:data[i]["ind_69"].find('</div></div>')]
        Citations_per_Paper=data[i]["ind_70"][45:data[i]["ind_70"].find('</div></div>')]
        Academic_Reputation=data[i]["ind_76"][45:data[i]["ind_76"].find('</div></div>')]
        Employer_Reputation=data[i]["ind_77"][45:data[i]["ind_77"].find('</div></div>')]
        temp=pd.DataFrame([[Overall_Rank,Overall_Rank_Dis,University,Region,Location,City,Overall_Score,H_index_Citations,Citations_per_Paper,Academic_Reputation,Employer_Reputation]],columns=columns_name)
        #print(Region,Location,City,Overall_Rank,University,Overall_Score,International_Students_Ratio_Score,International_Faculty_Ratio_Score,Faculty_Student_Ratio_Score,Citations_per_Faculty_Score,Academic_Reputation_Score,Employer_Reputa_Score)

        all_school=all_school.append(temp)
    return all_school

In [9]:
#從QS大學排名網站取得資料來源(get_json_web())後，抓取原始json檔，搜索資料____畢業生就業排名
def get_GER_rank(year):
    web="https://www.topuniversities.com/university-rankings/employability-rankings/"+str(year)+"?utm_source=topnav"
    json_web=get_json_web(web)
    
    jsonData=requests.get(json_web)
    data=json.loads(jsonData.text)
    data=data['data']
    #print(json.dumps(data,indent=2,ensure_ascii=False))
    all_school=pd.DataFrame()
    columns_name=["OverallRank","OverallRank_Dis","University","Region","Location","City","Overall Score","Graduate Employment Rate","Partnerships with Employers","Employer Reputation","Alumni Outcomes","Employer-Student Connections"]
    
    #print(data)
    for i in range(0,len(data)):
        Region=data[i]["region"]
        Location=data[i]["location"]
        City=data[i]["city"]
        Overall_Rank=data[i]["overall_rank"]
        Overall_Rank_Dis=data[i]["overall_rank_dis"][45:data[i]["overall_rank_dis"].find('</div></div>')]
        University=data[i]["uni"][data[i]["uni"].find('" class="uni-link">')+19:data[i]["uni"].find('</a>')]
        Overall_Score=data[i]["overall"][45:data[i]["overall"].find('</div></div>')]
        
        Graduate_Employment_Rate=data[i]["ind_19"][45:data[i]["ind_19"].find('</div></div>')]
        Partnerships_with_Employers=data[i]["ind_9"][45:data[i]["ind_9"].find('</div></div>')]
        Employer_Reputation=data[i]["ind_77"][45:data[i]["ind_77"].find('</div></div>')]
        Alumni_Outcomes=data[i]["ind_33"][45:data[i]["ind_33"].find('</div></div>')]
        Employer_Student_Connections=data[i]["ind_25"][45:data[i]["ind_25"].find('</div></div>')]
        
        temp=pd.DataFrame([[Overall_Rank,Overall_Rank_Dis,University,Region,Location,City,Overall_Score,Graduate_Employment_Rate,Partnerships_with_Employers,Employer_Reputation,Alumni_Outcomes,Employer_Student_Connections]],columns=columns_name)
        #print(Region,Location,City,Overall_Rank,University,Overall_Score,International_Students_Ratio_Score,International_Faculty_Ratio_Score,Faculty_Student_Ratio_Score,Citations_per_Faculty_Score,Academic_Reputation_Score,Employer_Reputa_Score)

        all_school=all_school.append(temp)
    return all_school

In [23]:
#大學總排名
writer = pd.ExcelWriter('./QS大學排名(2019-2022).xlsx')

for year in range(2019,2023):
    all_school=get_rank(year).to_excel(writer,sheet_name = str(year),index=False)
writer.save()

In [ ]:
#大學基本資料&師生資訊
BaseInfo_All=pd.DataFrame()
Stundent_Staff_Info_All=pd.DataFrame()
index=0
all_school=pd.read_excel('2021QS大學排名.xlsx')
for uni in all_school['Uni_Herf']:
    #print(uni)
    try:
        BaseInfo_All=BaseInfo_All.append(get_BaseInfo(uni))
    except:
        print(uni,"BaseInfo Error")
    
    try:
        Stundent_Staff_Info_All=Stundent_Staff_Info_All.append(get_Stundent_Staff_Info2(uni))
    except:
        print(uni,"Stundent_Staff_Info Error")
    index=index+1
    if index %100==0:
        print(index)
#BaseInfo_All.to_excel('QS排名大學基本資料.xlsx',index=False)
Stundent_Staff_Info_All.to_excel('QS排名大學師生資料.xlsx',index=False)

In [13]:
#學科排名
all_sub=pd.read_excel('sub.xlsx')#sub.xlsx中含有所有學排名網站
for sub,topic,web2017,web2018,web2019,web2020,web2021 in zip(all_sub['sub'],all_sub['topic'],all_sub['2017web'],all_sub['2018web'],all_sub['2019web'],all_sub['2020web'],all_sub['2021web']):
    # print('** ('+sub+' Start **')
    if sub==topic:
        writer = pd.ExcelWriter('./QS學科排名/QS大學學科排名_'+topic+'.xlsx')
    else:
        writer = pd.ExcelWriter('./QS學科排名/QS大學學科排名_'+topic+'_'+sub+'.xlsx')
    #2017 
    try:
        get_sub_rank(web2017).to_excel(writer,sheet_name = '2017',index=False)
    except:
        print(sub,"2017 Error")
    #2018
    try:
        get_sub_rank(web2018).to_excel(writer,sheet_name = '2018',index=False)
    except:
        print(sub,"2018 Error")
    #2019
    try:
        get_sub_rank(web2019).to_excel(writer,sheet_name = '2019',index=False)
    except:
        print(sub,"2019 Error")
    #2020 
    try:
        get_sub_rank(web2020).to_excel(writer,sheet_name = '2020',index=False)
    except:
        print(sub,"2020 Error")
    #2021
    try:
        get_sub_rank(web2021).to_excel(writer,sheet_name = '2021',index=False)
    except:
        print(sub,"2021 Error")
    
    writer.save()
    print('<< ('+sub+' Clear >>')

<< (Arts and Humanities Clear >>
<< (Archaeology Clear >>
Architecture and Built Environment 2017 Error
<< (Architecture and Built Environment Clear >>
Art and Design 2017 Error
<< (Art and Design Clear >>
Classics and Ancient History 2017 Error
<< (Classics and Ancient History Clear >>
English Language and Literature 2017 Error
<< (English Language and Literature Clear >>
<< (History Clear >>
<< (Linguistics Clear >>
Modern Languages 2017 Error
<< (Modern Languages Clear >>
Performing Arts 2017 Error
<< (Performing Arts Clear >>
<< (Philosophy Clear >>
<< (Theology, Divinity and Religious Studies Clear >>
<< (Engineering and Technology Clear >>
<< (Computer Science and Information Systems Clear >>
<< (Engineering - Chemical Clear >>
<< (Engineering - Civil and Structural Clear >>
<< (Engineering - Electrical and Electronic Clear >>
<< (Engineering - Mechanical Clear >>
<< (Engineering - Mineral and Mining Clear >>
Engineering - Petroleum 2017 Error
Engineering - Petroleum 2018 Error
E

In [9]:
#畢業生就業力排名
writer = pd.ExcelWriter('./QS畢業生就業力排名2021.xlsx')
for year in range(2022,2023):
    get_GER_rank(year).to_excel(writer,sheet_name = str(year),index=False)
writer.save()

In [9]:
#從QS大學排名網站取得資料來源(get_json_web())後，抓取原始json檔，搜索資料____大學總排名
def get_asia_rank(year):
#Rank
    url=get_json_web('https://www.topuniversities.com/university-rankings/asian-university-rankings/'+str(year))
    
    jsonData=requests.get(url)
    data=json.loads(jsonData.text)
    data=data['data']
    #print(data)
    #print(json.dumps(data,indent=2,ensure_ascii=False))
   
    all_school=pd.DataFrame()
    columns_name=["Overall Rank","OverallRank_Dis","University","Region","Location","City","Overall Score","International_Research_Network","Faculty_Student_Ratio","International_Students","Outbound_Exchange","Citations_per_Paper","Inbound_Exchange","Papers_per_Faculty","International_Faculty","Academic_Reputation","Employer_Reputation","Faculty_Staff_with_PhD"]
    #if year<=2019:
    #    columns_name=["Overall Rank","OverallRank_Dis","University","Region","Location","City","Overall Score","International_Research_Network","Faculty_Student_Ratio","International_Students","Outbound_Exchange","Citations_per_Paper","Inbound_Exchange","Papers_per_Faculty","International_Faculty","Academic_Reputation","Employer_Reputation","Faculty_Staff_with_PhD"]
    
    for i in range(0,len(data)):
        #print(i)
        Region=data[i]["region"]
        Location=data[i]["location"]
        City=data[i]["city"]
        Overall_Rank=data[i]["overall_rank"]
        University=data[i]["uni"][data[i]["uni"].find('" class="uni-link">')+19:data[i]["uni"].find('</a>')]
        Uni_Herf=data[i]["uni"][68:data[i]["uni"].find('" class="uni-link">')]
        Overall_Score=data[i]["overall"][45:data[i]["overall"].find('</div></div>')]
        Overall_Rank_Dis=data[i]["overall_rank_dis"][45:data[i]["overall_rank_dis"].find('</div></div>')]
        if year==2018:
            International_Research_Network='NaN'
        else:
            International_Research_Network=data[i]["ind_15"][45:data[i]["ind_15"].find('</div></div>')]
        if year<=2019:
            Faculty_Student_Ratio=data[i]["ind_74"][45:data[i]["ind_74"].find('</div></div>')]
        else:
            Faculty_Student_Ratio=data[i]["ind_36"][45:data[i]["ind_36"].find('</div></div>')]
        International_Students=data[i]["ind_37"][45:data[i]["ind_37"].find('</div></div>')]
        Outbound_Exchange=data[i]["ind_66"][45:data[i]["ind_66"].find('</div></div>')]
        Citations_per_Paper=data[i]["ind_70"][45:data[i]["ind_70"].find('</div></div>')]
        Inbound_Exchange=data[i]["ind_71"][45:data[i]["ind_71"].find('</div></div>')]
        Papers_per_Faculty=data[i]["ind_72"][45:data[i]["ind_72"].find('</div></div>')]
        International_Faculty=data[i]["ind_75"][45:data[i]["ind_75"].find('</div></div>')] 
        Academic_Reputation=data[i]["ind_76"][45:data[i]["ind_76"].find('</div></div>')]
        Employer_Reputation=data[i]["ind_77"][45:data[i]["ind_77"].find('</div></div>')]
        Faculty_Staff_with_PhD=data[i]["ind_101"][45:data[i]["ind_101"].find('</div></div>')]
        
        temp=pd.DataFrame([[Overall_Rank,Overall_Rank_Dis,University,Region,Location,City,Overall_Score,International_Research_Network,Faculty_Student_Ratio,International_Students,Outbound_Exchange,Citations_per_Paper,Inbound_Exchange,Papers_per_Faculty,International_Faculty,Academic_Reputation,Employer_Reputation,Faculty_Staff_with_PhD]],columns=columns_name)
        
        all_school=all_school.append(temp)

    #print(all_school)
    return all_school

In [53]:
#年輕大學排名
def get_fastest_rising_rank(year):
#Rank
    if year<2018:
        url=get_json_web("https://www.topuniversities.com/university-rankings/top-50-under-50/"+str(year))
    elif year==2018:
        url="https://www.qschina.cn/sites/default/files/qs-rankings-data/en/360008.txt?_=1636007077042"
    elif year==2019:
        url="https://www.qschina.cn/sites/default/files/qs-rankings-data/en/399336.txt?_=1636006831536"
        
    jsonData=requests.get(url)
    data=json.loads(jsonData.text)
    data=data['data']
    #print(data)
    #print(json.dumps(data,indent=2,ensure_ascii=False))
   
    all_school=pd.DataFrame()
    columns_name=["Overall Rank","OverallRank_Dis","University","Region","Location","City","Overall Score","Establishment Year","WUR Overall"]
    
    for i in range(0,len(data)):
        Region=data[i]["region"]
        if year<2018:
            Location=data[i]["location"]
            City=data[i]["city"]
            Overall_Rank=data[i]["overall_rank"]
            University=data[i]["uni"][data[i]["uni"].find('" class="uni-link">')+19:data[i]["uni"].find('</a>')]
            Uni_Herf=data[i]["uni"][68:data[i]["uni"].find('" class="uni-link">')]
            Overall_Score=data[i]["overall"][45:data[i]["overall"].find('</div></div>')]
            Overall_Rank_Dis=data[i]["overall_rank_dis"][45:data[i]["overall_rank_dis"].find('</div></div>')]
            Establishment_Year=data[i]["ind_53"][45:data[i]["ind_53"].find('</div></div>')]
            WUR_Overall=data[i]["ind_39"][45:data[i]["ind_39"].find('</div></div>')]
        elif year<2020:
            Location=data[i]["country"]
            City=""
            Overall_Rank=data[i]["rank_display"]
            University=data[i]["title"][data[i]["title"].find('" class="uni-link">')+19:data[i]["title"].find('</a>')]
            Uni_Herf=""
            Overall_Score=data[i]["score"]
            Overall_Rank_Dis=data[i]["rank_display"]
            Establishment_Year=data[i]["ind_0"][21:data[i]["ind_0"].find('</div>')]
            WUR_Overall=data[i]["ind_1"][21:data[i]["ind_1"].find('</div>')]
        temp=pd.DataFrame([[Overall_Rank,Overall_Rank_Dis,University,Region,Location,City,Overall_Score,Establishment_Year,WUR_Overall]],columns=columns_name)
        
        all_school=all_school.append(temp)

    #print(all_school)
    return all_school

In [10]:
#大學總排名
writer = pd.ExcelWriter('./QS亞洲大學排名(2018-2022).xlsx')

for year in range(2018,2023):
    all_school=get_asia_rank(year).to_excel(writer,sheet_name = str(year),index=False)
writer.save()

In [54]:
#年輕大學排名
writer = pd.ExcelWriter('./QS年輕大學排名(2014-2019).xlsx')

for year in range(2014,2020):
    all_school=get_fastest_rising_rank(year).to_excel(writer,sheet_name = str(year),index=False)
writer.save()
    

In [17]:
def get_QS_SDG_ranking():
    url='https://www.topuniversities.com/sites/default/files/qs-rankings-data/en/3740566_medals.txt?1636438899?v=1636446472566'
    jsonData=requests.get(url)
    data=json.loads(jsonData.text)
    data=data['data']
        #print(data)
    #print(json.dumps(data,indent=2,ensure_ascii=False))
    all_school=pd.DataFrame()
    columns_name=["Rank","University","Region","Country","City","Environmental Impact","Equal Opportunities"]
    
    for i in range(0,len(data)):
        Rank=data[i]["rank_display"]
        University=data[i]["title"][data[i]["title"].find('" class="uni-link">')+19:data[i]["title"].find('</a>')]
        Region=data[i]["region"]
        Country=data[i]["country"]
        City=data[i]["city"]
        Environmental_Impact=data[i]["medal_env_impact"]
        Equal_Opportunities=data[i]["medal_equal_opp"]
        
        temp=pd.DataFrame([[Rank,University,Region,Country,City,Environmental_Impact,Equal_Opportunities]],columns=columns_name)
        
        all_school=all_school.append(temp)

    #print(all_school)
    return all_school

In [18]:
writer = pd.ExcelWriter('./QS_SDG大學排名(2022).xlsx')

get_QS_SDG_ranking().to_excel(writer,sheet_name = str(2022),index=False)
writer.save()